In [1]:
# pip install --user --upgrade git+https://github.com/himanshudabas/twint.git@origin/twint-fixes#egg=twint

In [2]:
import numpy as np
import pandas as pd
import twint
import time
# from datetime import datetime
from tqdm import tqdm

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

# PATHS

In [3]:
# split = 0

In [4]:
THIS_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/FINAL USER BIO PULLING/Final Splits for Pulling Bios/Split 0 - MIKHAEL.csv"

# SAVE_SPLIT_PATH = fr"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/User Account Info Lookup/Searched User Results/Split {split}.csv"

# Import Split List

In [5]:
local_split_users = pd.read_csv(THIS_SPLIT_PATH)

In [6]:
cols_to_return = ['location', 'following', 'followers', 'likes', 'tweets', 'private', 'verified', 'join_date']

In [7]:
cols_to_overwrite = cols_to_return + ['Searched - Bio', 'Error - Bio']

In [8]:
#create new columns if they aren't there yet
if not all(x in local_split_users.columns.tolist() for x in cols_to_overwrite):
    local_split_users[cols_to_overwrite] = np.nan

# Define Lookup Function

In [9]:
def get_user_info(ID):
    try:
        c = twint.Config()
        c.User_id = ID
        c.Hide_output = True
        c.Pandas = True

        twint.run.Lookup(c)
        return twint.storage.panda.User_df[cols_to_return].values.tolist()[0]
    except:
        return [np.nan]*len(cols_to_return)

# Apply Lookup Function

In [10]:
# DELETE THIS
# local_split_users = MASTER_USER_LIST.head()

In [11]:
local_split_users = local_split_users.set_index('ID')

In [ ]:
# user_lookup_dict = {}

for i, ID in enumerate(local_split_users.index.astype(str).tolist()):
    if local_split_users.loc[int(ID), 'Searched - Bio'] != 1:
        print(i, ID)
        if i % 100 == 0:
            try:
                local_split_users.loc[int(ID), cols_to_overwrite] = get_user_info(ID) + [1, 0]
    #             user_lookup_dict[ID] = get_user_info(ID)
            except:
                local_split_users.loc[int(ID), ['Searched - Bio', 'Error - Bio']] = [1, 1]
    #             user_lookup_dict[ID] = [np.nan]*len(cols_to_return)
            local_split_users.to_csv(THIS_SPLIT_PATH)


        else:
            try:
                local_split_users.loc[int(ID), cols_to_overwrite] = get_user_info(ID) + [1, 0]        
            except:
                local_split_users.loc[int(ID), ['Searched - Bio', 'Error - Bio']] = [1, 1]

0 1041624996265701378
1 1279658870
2 2901430149
3 280508095
4 94250223
5 2413582404
6 1159073503267758080
7 153151535
8 859179698256924672
9 289186412
10 2251633075
11 1665847111
12 94374562
13 1184345781353844736
14 21123733
15 961623365499936769
16 1001471784460476416
17 1106312355267469312
18 1212596361985544192
19 4757003540
20 1001110560728068096
21 4440362292
22 1489567152
23 985965515482025984
24 821546923035197440
25 568614476
26 2601765193
27 706860096232357888
28 1043174069808623622
29 883444040280137728
30 2577620557
31 1039792010339602433
32 951357527501008896
33 1047222531579686912
34 821149607048646656
35 28693346
36 979053160785301505
37 1189844650749763584
